## Анализ демографических и экономических показателей регионов России

### __Введение__
#### В современном мире мы часто слышим новости о различных проблемах с демографией в России. Давайте разберемся вместе, в чем может крыться причина распространения таких новостей, и правда ли всё так плохо?

Данный отчет описывает разработку программы для анализа демографических данных регионов России, полученных из открытых источников. 

__Программа позволяет__:

- загружать и объединять данные из разных файлов
- рассчитывать средние значения по ключевым показателям
- выявлять регионы с наибольшими демографическими рисками
- анализировать среднее количество рождений в регионах

### Описание
Данные для анализа были взяты с сайта __Росстата__ (https://rosstat.gov.ru/) и преобразованы вручную в текстовые файлы с разделителями-запятыми. 

__Имена файлов:__

1. dohod2021.txt: содержит медианные доходы населения по регионам в 2021 году
2. bezrab2021.txt: содержит количество безработных по регионам в 2021 году
3. migration2021.txt: содержит данные о миграции по регионам в 2021 году
4. rozhdenie2021.txt: содержит данные о рождаемости и смертности по регионам в 2021 году

### Структура файлов

Каждый файл имеет следующий __формат__:

In [2]:
<Регион>, <Значение>

SyntaxError: invalid syntax (4220206845.py, line 1)

Например, файл dohod2021.txt может содержать:

In [ ]:
Москва,50000
Санкт-Петербург,45000
...

### Описание программы

Программа написана на языке Python и состоит из нескольких __функций__:

#### __1.__ load_data(filename):

- Загружает данные из файла в список списков
- Каждый внутренний список представляет строку данных
- Преобразует значения в нужный тип данных

__Пример кода:__

In [2]:
def load_data(filename):
    """
    Загружает данные из файла в список списков.

    Args:
      filename: имя файла.

    Returns:
      Список списков, где каждый внутренний список представляет строку данных.
    """
    data = []
    with open(filename, 'r', encoding='utf-8') as f:
        for line in f:
            parts = line.strip().split(",")
            data.append(parts)
    return data

#### __2.__ merge_data(dohod, bezrab, migration, rozhdenie):

- Объединяет данные из всех файлов в один список списков
- Сопоставляет данные по регионам, используя индекс

__Пример кода:__

In [3]:
def merge_data(dohod, bezrab, migration, rozhdenie):
    """
    Объединяет данные из всех файлов в один список списков.

    Args:
      dohod: Список списков с медианными доходами.
      bezrab: Список списков с количеством безработных.
      migration: Список списков с данными о миграции.
      rozhdenie: Список списков с данными о рождаемости и смертности.

    Returns:
      Список списков с объединенными данными.
    """
    data = []
    for i in range(len(dohod)):
        region = dohod[i][0]
        dohod_value = int(dohod[i][1])
        bezrab_value = int(bezrab[i][1])
        migration_value = float(migration[i][3])  # Используем четвертый столбец
        rozhdenie_value = float(rozhdenie[i][3])  # Используем четвертый столбец
        # Добавляем количество родившихся во второй столбец rozhdenie
        rozhdenie_count = int(rozhdenie[i][1]) 
        data.append([region, dohod_value, bezrab_value, migration_value, rozhdenie_value, rozhdenie_count])
    return data

#### __3.__ calculate_averages(data):

Рассчитывает средние значения для всех показателей:

- средний доход
- средний уровень безработицы
- средний миграционный прирост
- средний естественный прирост
- среднее количество рождений

__Пример кода:__

In [4]:
def calculate_averages(data):
    """
    Рассчитывает средние значения для всех показателей.

    Args:
      data: Список списков с объединенными данными.

    Returns:
      Словарь со средними значениями.
    """
    averages = {}
    total_dohod = 0
    total_bezrab = 0
    total_migration = 0
    total_rozhdenie = 0
    total_rozhdenie_count = 0
    for row in data:
        total_dohod += row[1]
        total_bezrab += row[2]
        total_migration += row[3]
        total_rozhdenie += row[4]
        total_rozhdenie_count += row[5]

    averages["Средний доход"] = total_dohod / len(data)
    averages["Средний уровень безработицы"] = total_bezrab / len(data)
    averages["Средний миграционный прирост"] = total_migration / len(data)
    averages["Средний естественный прирост"] = total_rozhdenie / len(data)
    averages["Среднее количество рождений"] = total_rozhdenie_count / len(data)
    return averages

#### __4.__ find_risk_regions(data, average_dohod):

Находит регионы с наибольшими демографическими рисками, т.е. регионы с низким доходом и отрицательным естественным приростом.

__Пример кода:__

In [5]:
def find_risk_regions(data, average_dohod):
    """
    Находит регионы с наибольшими демографическими рисками.

    Args:
      data: Список списков с объединенными данными.
      average_dohod: Средний доход.

    Returns:
      Список списков с регионами, имеющими низкий доход и отрицательный естественный прирост.
    """
    risk_regions = []
    for row in data:
        if row[1] < average_dohod and row[4] < 0:
            risk_regions.append(row)
    return risk_regions

#### Далее будет приведен __пример__ использования программы, способной посчитать средние значения для различных показателей и выявить необходимые риски, используя выше введенные __формулы__

In [6]:
if __name__ == "__main__":
    dohod = load_data("data/dohod2021.txt")
    bezrab = load_data("data/bezrab2021.txt")
    migration = load_data("data/migration2021.txt")
    rozhdenie = load_data("data/rozhdenie2021.txt")

    data = merge_data(dohod, bezrab, migration, rozhdenie)

    averages = calculate_averages(data)
    print("Средние значения:")
    for key, value in averages.items():
        print(f"{key}: {value:.2f}")

    average_dohod = averages["Средний доход"]
    risk_regions = find_risk_regions(data, average_dohod)
    print("\nРегионы с наибольшими демографическими рисками:")
    if risk_regions:
        for region in risk_regions:
            print(f"Регион: {region[0]}, Доход: {region[1]}, Естественный прирост: {region[4]}")
    else:
        print("В данных нет регионов с наибольшими демографическими рисками.")

Средние значения:
Средний доход: 28363.99
Средний уровень безработицы: 43662.44
Средний миграционный прирост: -4.67
Средний естественный прирост: -8358.00
Среднее количество рождений: 17101.36

Регионы с наибольшими демографическими рисками:
Регион: ﻿Белгородская область, Доход: 27986, Естественный прирост: -11745.0
Регион: Брянская область, Доход: 25014, Естественный прирост: -10637.0
Регион: Владимирская область, Доход: 23376, Естественный прирост: -14608.0
Регион: Воронежская область, Доход: 26828, Естественный прирост: -19173.0
Регион: Ивановская область, Доход: 24269, Естественный прирост: -10034.0
Регион: Калужская область, Доход: 26978, Естественный прирост: -8352.0
Регион: Костромская область, Доход: 25093, Естественный прирост: -5136.0
Регион: Курская область, Доход: 26143, Естественный прирост: -10285.0
Регион: Липецкая область, Доход: 27146, Естественный прирост: -10721.0
Регион: Орловская область, Доход: 23979, Естественный прирост: -7667.0
Регион: Рязанская область, Доход: